In [1]:
import nb_utils

Moving cwd to /store/CIA/js2164/repos/diffusion/score_sde_pytorch


In [2]:
import torch
import numpy as np
import pandas as pd
import pickle
from matplotlib import pyplot as plt
from mle import bootstrap_intrinsic_dim_scale_interval as mle

In [3]:
#Import R packages
import rpy2
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri
pandas2ri.activate()

from rpy2.robjects.packages import importr
r_base = importr('base')
intdimr = importr('intrinsicDimension')

In [4]:
from lightning_data_modules import HaarDecomposedDataset, ImageDatasets, PairedDataset, SyntheticDataset, SyntheticPairedDataset, Synthetic1DConditionalDataset, SyntheticTimeSeries, SRDataset, SRFLOWDataset, KSphereDataset, MammothDataset, LineDataset, GanDataset #needed for datamodule registration

#from configs.mammoth.vesde import get_config
#from configs.ksphere.N_1.uniform import get_config
#from configs.line.vesde import get_config
#from configs.ksphere.N_1.non_uniform_1 import get_config
#from configs.ksphere.N_1.non_uniform_075 import get_config
#from configs.ksphere.N_1.non_uniform_05 import get_config
from configs.utils import read_config

config = read_config('configs/fixedgaussiansmanifold/10_small.py')

In [5]:
from lightning_data_modules.utils import create_lightning_datamodule
DataModule = create_lightning_datamodule(config)
DataModule.setup()
train_dataloader = DataModule.train_dataloader()

In [ ]:
with open('small_gaussian_maifold_dataloader.pkl', 'wb') as f:
    pickle.dump(train_dataloader, f)

In [ ]:
X=[]
for _, x in enumerate(train_dataloader):
    X.append(x.view(x.shape[0],-1))
X = torch.cat(X, dim=0).numpy()
X = pd.DataFrame(X)

In [ ]:
print(X.shape)
X.head()

(4000, 1024)


,0,1,2,3,4,5,6,7,8,9,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# MLE

In [14]:
k = 5
mle_result = mle(X,  k1=k, k2=k, average=True)[0]

KeyboardInterrupt: 

In [13]:
mle_result

NameError: name 'mle_result' is not defined

# MLE R

In [26]:
data_r = X.to_numpy()
dim = X.shape[1]
k=5
#mle = intdimr.maxLikPointwiseDimEst(data_r, k=5).rx2('dim.est')
mle = intdimr.maxLikGlobalDimEst(data_r, k=k).rx2('dim.est')

In [27]:
mle

array([32.79388876])

# Local PCA

In [28]:
data_r = X
dim = X.shape[1]
lpca = intdimr.pcaLocalDimEst(data_r, 'FO').rx2('dim.est')

In [29]:
lpca

array([72], dtype=int32)

In [9]:
from sklearn.decomposition import PCA
data_r = X.to_numpy().astype(np.float64)
pca = PCA(n_components='mle')
pca.fit(data_r)
pca.n_components_


99

In [46]:
data_r + 1e-12

array([[1.e-12, 1.e-12, 1.e-12, ..., 1.e-12, 1.e-12, 1.e-12],
       [1.e-12, 1.e-12, 1.e-12, ..., 1.e-12, 1.e-12, 1.e-12],
       [1.e-12, 1.e-12, 1.e-12, ..., 1.e-12, 1.e-12, 1.e-12],
       ...,
       [1.e-12, 1.e-12, 1.e-12, ..., 1.e-12, 1.e-12, 1.e-12],
       [1.e-12, 1.e-12, 1.e-12, ..., 1.e-12, 1.e-12, 1.e-12],
       [1.e-12, 1.e-12, 1.e-12, ..., 1.e-12, 1.e-12, 1.e-12]],
      dtype=float32)

In [49]:
data_r

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)